In [4]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [5]:
INPUT_DIR = "../../../graphrag/output"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [6]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 2886


,id,human_readable_id,title,community,level,degree,x,y
0,45ecb081-6acc-4f25-a98a-57b99b026508,0,AUDI S3 SEDÁN TFSI QUATTRO,7,0,24,0.0,0.0
1,45ecb081-6acc-4f25-a98a-57b99b026508,0,AUDI S3 SEDÁN TFSI QUATTRO,51,1,24,0.0,0.0
2,45ecb081-6acc-4f25-a98a-57b99b026508,0,AUDI S3 SEDÁN TFSI QUATTRO,118,2,24,0.0,0.0
3,45ecb081-6acc-4f25-a98a-57b99b026508,0,AUDI S3 SEDÁN TFSI QUATTRO,150,3,24,0.0,0.0
4,4b1d0dac-3cb8-48eb-b9ad-8f1f2489e6e3,1,CUATRO CILINDROS EN LÍNEA,7,0,7,0.0,0.0


In [7]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()


Relationship count: 1177


,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,446b3a2a-a0cf-43d2-8ba6-6789ccb2420e,0,AUDI S3 SEDÁN TFSI QUATTRO,CUATRO CILINDROS EN LÍNEA,El motor de cuatro cilindros en línea es el mo...,9.0,31,[17a36093436dba59f919711728dfbc006ee25724db974...
1,66b5148e-9ae0-4d5d-9d57-0fa452f4e078,1,AUDI S3 SEDÁN TFSI QUATTRO,S-TRONIC DE 7 CAMBIOS,La transmisión S-tronic de 7 cambios es la que...,9.0,28,[17a36093436dba59f919711728dfbc006ee25724db974...
2,05df256d-465d-4b79-972e-6389fc951413,2,AUDI S3 SEDÁN TFSI QUATTRO,LONGITUD: 4.504 MM,La longitud del Audi S3 Sedán TFSI quattro es ...,7.0,26,[17a36093436dba59f919711728dfbc006ee25724db974...
3,a9a8692d-5a61-4012-ba35-9cb5aa842519,3,AUDI S3 SEDÁN TFSI QUATTRO,ANCHO CON ESPEJOS: 1.984 MM,El ancho del Audi S3 Sedán TFSI quattro con es...,7.0,29,[17a36093436dba59f919711728dfbc006ee25724db974...
4,9fc6ea38-5bae-4b68-b9ae-b97060002949,4,AUDI S3 SEDÁN TFSI QUATTRO,ALTO: 1.415 MM,La altura del Audi S3 Sedán TFSI quattro es de...,7.0,26,[17a36093436dba59f919711728dfbc006ee25724db974...


In [8]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 151


,id,human_readable_id,community,parent,level,title,summary,full_content,rank,rank_explanation,findings,full_content_json,period,size
0,5a38a3da217849549cec5111c34b3770,149,149,118,3,Audi S3 Sedán TFSI quattro y sus Especificaciones,La comunidad se centra en el Audi S3 Sedán TFS...,# Audi S3 Sedán TFSI quattro y sus Especificac...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El Audi S3 Sedán TFSI quattr...,"{\n ""title"": ""Audi S3 Sedán TFSI quattro y ...",2025-02-03,5
1,a2d7d2586a02492fb895d73505e90b4a,150,150,118,3,Audi S3 Sedán TFSI Quattro y Anclajes ISOFIX,La comunidad se centra en el Audi S3 Sedán TFS...,# Audi S3 Sedán TFSI Quattro y Anclajes ISOFIX...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'Los anclajes para asientos i...,"{\n ""title"": ""Audi S3 Sedán TFSI Quattro y ...",2025-02-03,5
2,45d1aaaf7dac4cdf804a039250bb7ad5,97,97,33,2,Análisis del Renault Kwid y sus Sistemas de Se...,"La comunidad se centra en el Renault Kwid, des...",# Análisis del Renault Kwid y sus Sistemas de ...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El Control Electrónico de Es...,"{\n ""title"": ""Análisis del Renault Kwid y s...",2025-02-03,3
3,31a6e569150640268229ffb8a64c24ef,98,98,33,2,Audi Q8 55 TFSI quattro MHEV y Sistemas de Seg...,La comunidad se centra en el Audi Q8 55 TFSI q...,# Audi Q8 55 TFSI quattro MHEV y Sistemas de S...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El Sistema de Control de Tra...,"{\n ""title"": ""Audi Q8 55 TFSI quattro MHEV ...",2025-02-03,3
4,0156b5e383734ea1b2402050ac21f2d8,99,99,33,2,Audi Q8 55 TFSI quattro MHEV y Sistemas de Seg...,La comunidad se centra en el Audi Q8 55 TFSI q...,# Audi Q8 55 TFSI quattro MHEV y Sistemas de S...,9.0,La calificación es alta debido a la relevancia...,[{'explanation': 'El Control de Tracción (ASR)...,"{\n ""title"": ""Audi Q8 55 TFSI quattro MHEV ...",2025-02-03,4


In [9]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 82


,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,17a36093436dba59f919711728dfbc006ee25724db9748...,1,Audi S3 Sedán TFSI quattro - Ficha Técnica Com...,949,[0176fe4f3cb16e417b15b36f7797b9e0e16dbe4d25230...,"[45ecb081-6acc-4f25-a98a-57b99b026508, 4b1d0da...","[446b3a2a-a0cf-43d2-8ba6-6789ccb2420e, 66b5148..."
1,f7f4a3b50ed695879b783902a1fd09f49195069e75c93b...,2,**Brand**: *Renault* \n**Model**: *Kwid E-Tec...,1200,[080ce8e2d1d1092f59b9ac5ae06f19fdeccc69e8afa40...,"[ef26aaf9-ca56-4041-84e4-effc03cd5599, 17bf4fb...","[af855e08-945c-495b-9bf3-1c100529be97, d8afe12..."
2,3b30cd506f4492e0980a9a31afb415529ebf9939a23ec5...,3,**Funciones**: Modo ECO para maximizar la aut...,680,[080ce8e2d1d1092f59b9ac5ae06f19fdeccc69e8afa40...,"[6b25dd49-3caf-41eb-8a1b-c4bd0c39a9b1, c5c8af5...","[c5e80381-db28-45fc-8901-7ae3dcae3f1f, 3e02464..."
3,53fbf6c77a74e72173b94b3193ad7fd8eabd00d0a3f624...,4,Audi RS3 Sportback - Ficha Técnica Completa - ...,1076,[08c17ef4280f76430be642cfe6f21c49f6bc2585abb84...,"[8d873304-dc34-421d-8370-c1bc108f250c, c85c149...","[fe2d081a-4177-42ef-9bcb-237d64058600, 6af7892..."
4,e2f5289b98a949a0ec3c10b465a6b397625cdbe74c479f...,5,Audi Q3 Sportback 35 TFSI Ambition Plus - Fich...,1143,[12ebcd4306a0e57db19da9932457320cb891d07fc5edf...,"[4b1d0dac-3cb8-48eb-b9ad-8f1f2489e6e3, 5cd2b39...","[90a27bcc-f368-4f29-9fb7-f2fcaaed90fe, de3c0ca..."


In [10]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

In [11]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [12]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [13]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [15]:
result = await search_engine.asearch("Carros electricos por menos de 200 millones")
print(result.response)

En el mercado actual, los vehículos eléctricos están ganando popularidad debido a su eficiencia energética y menor impacto ambiental. Sin embargo, encontrar opciones por debajo de los 200 millones de pesos puede ser un desafío, ya que muchos modelos eléctricos tienden a ser más costosos debido a la tecnología avanzada que incorporan.

### Renault Zoe E-Tech Iconic 2024

Una opción que se encuentra cerca de este rango de precio es el Renault Zoe E-Tech Iconic 2024, que tiene un precio de $149.990.000 [Data: Entities (820); Sources (23)]. Este modelo es un hatchback eléctrico que ofrece una autonomía de hasta 395 km en ciclo WLTP, lo que lo hace adecuado para desplazamientos urbanos y viajes cortos [Data: Sources (23)]. El Zoe E-Tech está equipado con un motor eléctrico de 135 HP y una batería de 52 kWh, lo que proporciona un buen equilibrio entre rendimiento y eficiencia [Data: Sources (23)].

### Renault Kwid E-Tech Intens 2024

Otra opción es el Renault Kwid E-Tech Intens 2024, que ti